# Decoding with SpaceNet 

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy as np
import nilearn
import glob
from nilearn.image import index_img


### Load the Chocolate data 
- Starting with W1 (133 Subjects)

In [ ]:
basepath=os.path.join('/projects','niblab','data','eric_data','W1','imagine')
#basepath ='/projects/niblab/nilearn_projects'
outpath = "/projects/niblab/nilearn_projects"
#make a list of the files to concat
all_func = glob.glob(os.path.join(basepath,'level1_grace_edit','cs*++.feat','filtered_func_data.nii.gz'))
len(all_func)

func_filenames=os.path.join(outpath, 'concatenated_imagine_all.nii')
average_ana=os.path.join(outpath,'CS_avg_mprage_image.nii.gz')
imag_mask=os.path.join(outpath,'power_roimask_4bi.nii.gz')
#imag_mask="/projects/niblab/data/eric_data/W1/imagine/level1_grace_edit/cs001++.feat/mask.nii.gz"
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
behavioral = pd.read_csv(stim, sep=",")
conditions =  behavioral['label']
condition_mask = conditions.isin(['unapp', 'app', 'H2O'])

# Split data into train and test samples, using the chunks
condition_mask_train = (condition_mask) & (behavioral['sub'] <= 0)
condition_mask_test = (condition_mask) & (behavioral['sub']  > 0)


# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
X_train = index_img(func_filenames, condition_mask_train)
X_test = index_img(func_filenames, condition_mask_test)
y_train = conditions[condition_mask_train]
y_test = conditions[condition_mask_test]

# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(func_filenames)

### Fit SpaceNet with a Graph-Net penalty 

In [ ]:
from nilearn.decoding import SpaceNetClassifier
# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory="nilearn_cache", penalty='graph-net')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("Graph-net classification accuracy : %g%%" % accuracy)